In this notebook we re-clean the data based on what we learned during EDA:
<ul>
    <li> reset unknown values into "other" </li>
        <ul>
            <li> Education values 0, 5, and 6 reset to 4 </li>
            <li> Marriage value 0 reset to 3 </li>
    </ul>
    <li> rename PAY_0 as PAY_1 </li>
    <li> rename 'default payment next month' as Y </li>
    <li> add new columns that were found to be interesting in EDA </li>
        <ul>
            <li> Age grouped by 10's </li>
            <li> Average Payment History </li>
            <li> Age/log(credit limit) </li>
            <li> Average bill_amt </li>
            <li> bill_amt/credit limit </li>
            <li> pay_amt/log(credit limit) </li>
            <li> pay_amt/bill_amt </li>
            <li> number of times bill not paid duly </li>
    </ul>
    <li> save the new csv as clean_data_2.csv </li>
</ul>

A full description of the data set can be found in the file dataset_2_description.txt

## Import packages and data

In [1]:
#importing packages
import pandas as pd
import numpy as np

In [2]:
#import data
data=pd.read_csv("clean_data.csv")

## Helper functions and important shortcuts

In [3]:
pay_cols=['PAY_1', 'PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']
bill_amt_cols=['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5','BILL_AMT6']
pay_amt_cols=['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5','PAY_AMT6']

In [4]:
payperbill = ['PAY/BILL_1','PAY/BILL_2','PAY/BILL_3','PAY/BILL_4','PAY/BILL_5',]

In [5]:
def average(L):
    '''takes in a non-empty list of numbers and outputs the average of those numbers'''
    sum=0
    for item in L:
        sum=sum +item
    return(sum/len(L))

## Renaming columns

In [6]:
data=data.rename({'PAY_0':'PAY_1'}, axis='columns')

In [7]:
data=data.rename({'default payment next month': 'Y'}, axis='columns')

## Resetting unknown values to "other"

If education is 0, 5, or 6, reset to 4

In [8]:
data['EDUCATION'].value_counts()

2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64

In [9]:
for i in range(len(data)):
    if data.loc[i,'EDUCATION'] in [0,5,6]:
        data.at[i,'EDUCATION'] = 4

In [10]:
data['EDUCATION'].value_counts()

2    14030
1    10585
3     4917
4      468
Name: EDUCATION, dtype: int64

If marriage is 0, reset it to 3

In [11]:
data['MARRIAGE'].value_counts()

2    15964
1    13659
3      323
0       54
Name: MARRIAGE, dtype: int64

In [12]:
for i in range(len(data)):
    if data.loc[i,'MARRIAGE'] ==0:
        data.at[i,'MARRIAGE']= 3

In [13]:
data['MARRIAGE'].value_counts()

2    15964
1    13659
3      377
Name: MARRIAGE, dtype: int64

## Checking that all the values still make sense

In [14]:
print('The youngest person is: ' + str(data['AGE'].min()))
print('The oldest person is: ' + str(data['AGE'].max()))

The youngest person is: 21
The oldest person is: 79


In [15]:
for col in pay_cols:
    print('The minimum payment was: ' + str(data[col].min()))
    print('The maximum payment was: ' + str(data[col].max()))

The minimum payment was: -2
The maximum payment was: 8
The minimum payment was: -2
The maximum payment was: 8
The minimum payment was: -2
The maximum payment was: 8
The minimum payment was: -2
The maximum payment was: 8
The minimum payment was: -2
The maximum payment was: 8
The minimum payment was: -2
The maximum payment was: 8


In [16]:
for col in pay_amt_cols:
    print('The minimum payment was: ' + str(data[col].min()))
    print('The maximum payment was: ' + str(data[col].max()))

The minimum payment was: 0
The maximum payment was: 873552
The minimum payment was: 0
The maximum payment was: 1684259
The minimum payment was: 0
The maximum payment was: 896040
The minimum payment was: 0
The maximum payment was: 621000
The minimum payment was: 0
The maximum payment was: 426529
The minimum payment was: 0
The maximum payment was: 528666


In [17]:
for col in bill_amt_cols:
    print('The minimum bill was: ' + str(data[col].min()))
    print('The maximum bill was: ' + str(data[col].max()))

The minimum bill was: -165580
The maximum bill was: 964511
The minimum bill was: -69777
The maximum bill was: 983931
The minimum bill was: -157264
The maximum bill was: 1664089
The minimum bill was: -170000
The maximum bill was: 891586
The minimum bill was: -81334
The maximum bill was: 927171
The minimum bill was: -339603
The maximum bill was: 961664


Great, so we didn't gain any new entries from using the whole data set instead of just our training set!

## Adding a column for age grouped by 10's

In [18]:
age_by10 = []
for i in range(len(data)):
    for j in range(8):
        if data.iloc[i]['AGE'] in range(19+10*j+1, 30+10*j):
            age_by10.append(j)

data['AGE_BY10']=age_by10

## Adding a column for average Payment History

In [19]:
newcol = []
for i in range(len(data)):
    newcol.append(average([data.iloc[i][j] for j in pay_cols]))
    
data['AVE_PAY']= newcol

## Adding a column for ratio of age to (logarithm of) credit limit

In [20]:
data['AGE/LIM']=data['AGE']/np.log(data['LIMIT_BAL'])

## Adding a column for the average bill amount

In [21]:
newcol = []
for i in range(len(data)):
    newcol.append(average([data.iloc[i][j] for j in bill_amt_cols]))
    
data['AVE_BILL_AMT']= newcol

## Adding columns for ratio of bill to credit limit

In [22]:
#First, we add the monthly columns
for j in range(len(bill_amt_cols)):
    col_name='BILL/LIM'+str(j+1)
    data[col_name]=data[bill_amt_cols[j]]/data['LIMIT_BAL']

#Then, we add a column that gets the average of these
data['AVE_BILL_AMT/LIM']=data['AVE_BILL_AMT']/data['LIMIT_BAL']

## Adding a column for average payment amount

In [23]:
newcol = []
for i in range(len(data)):
    newcol.append(average([data.iloc[i][j] for j in pay_amt_cols]))
    
data['AVE_PAY_AMT']= newcol

## Adding columns for ratio of payment amount to (log of) credit limit

In [24]:
#First, we add the monthly columns
for j in range(len(pay_amt_cols)):
    col_name='PAY_AMT/LIM'+str(j+1)
    data[col_name]=data[pay_amt_cols[j]]/np.log(data['LIMIT_BAL'])

#Then, we add a column that gets the average of these
data['AVE_PAY_AMT/LIM']=data['AVE_PAY_AMT']/data['LIMIT_BAL']

## Adding columns for ratio of payment amount to bill amount

In [25]:
#Adding blank columns to fill in

for k in range(len(bill_amt_cols)-1):
    data[payperbill[k]]=np.nan
    
data['AVE_PAY/BILL']=np.nan

In [26]:
#Adjustment for people overpaying their bills - any paid in full bill is counted as ratio 1
#Pay_AMT(n) coorsponds to BILL_AMT(n+1)


for i in range(len(data)):
    
    P=[]
    B=[]
    
    for j in range(len(bill_amt_cols)-1):
        if data.iloc[i][bill_amt_cols[j+1]] <= 0:
            P.append(1)
            B.append(1)
        elif data.iloc[i][pay_amt_cols[j]] > data.iloc[i][bill_amt_cols[j+1]]:
            P.append(1)
            B.append(1)
        else:
            P.append(data.iloc[i][pay_amt_cols[j]])
            B.append(data.iloc[i][bill_amt_cols[j+1]])
    
    for k in range(len(bill_amt_cols)-1):
        #col_name = 'PAY/BILL_'+str(k+1)
        data.at[i, payperbill[k]] = P[k]/B[k]
    
    data.at[i, 'AVE_PAY/BILL']=average([P[k]/B[k] for k in range(len(bill_amt_cols)-1)])
    

## Adding a column for the number of months bill was not paid duly

In [27]:
newcol=[]
for i in range(len(data)):
    counter=0
    for col in pay_cols:
        if data.iloc[i][col]>0:
            counter=counter+1
    newcol.append(counter)
data['UNPAID_MONTHS']=newcol

In [28]:
data.head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT/LIM5,PAY_AMT/LIM6,AVE_PAY_AMT/LIM,PAY/BILL_1,PAY/BILL_2,PAY/BILL_3,PAY/BILL_4,PAY/BILL_5,AVE_PAY/BILL,UNPAID_MONTHS
0,1,20000,2,2,1,24,2,2,-1,-1,...,0.000000,0.000000,0.005742,0.000000,1.000000,1.000000,1.000000,1.000000,0.800000,2
1,2,120000,2,2,2,26,-1,2,0,0,...,0.000000,171.009641,0.006944,0.000000,0.372856,0.305623,0.289436,0.000000,0.193583,2
2,3,90000,2,2,2,34,0,0,0,0,...,87.661127,438.305635,0.020404,0.108220,0.110628,0.069779,0.066899,0.064313,0.083968,0
3,4,50000,2,2,1,37,0,0,0,0,...,98.800546,92.423336,0.027960,0.041465,0.040961,0.042382,0.037985,0.036180,0.039794,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,63.679678,62.755445,0.196830,0.352734,1.000000,0.477555,0.470072,0.036015,0.467275,0
5,6,50000,1,1,2,37,0,0,0,0,...,92.423336,73.938669,0.025907,0.043807,0.031506,0.033876,0.050971,0.049940,0.042020,0
6,7,500000,1,1,2,29,0,0,0,0,...,1047.829541,1049.353657,0.060253,0.133488,0.089886,0.070026,0.041902,0.029012,0.072863,0
7,8,100000,2,2,2,23,0,-1,-1,0,...,146.530958,133.936418,0.007985,1.000000,1.000000,0.000000,1.000000,1.000000,0.800000,0
8,9,140000,2,3,1,28,0,0,2,0,...,84.392475,84.392475,0.008049,0.236166,0.000000,0.035378,0.084796,0.268889,0.125046,1
9,10,20000,1,3,2,35,-2,-2,-2,-2,...,113.293423,0.000000,0.117742,1.000000,1.000000,1.000000,1.000000,0.080650,0.816130,0


## Saving the file

In [29]:
data.to_csv("clean_data_2.csv", index=False)

In [30]:
data.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'Y',
       'AGE_BY10', 'AVE_PAY', 'AGE/LIM', 'AVE_BILL_AMT', 'BILL/LIM1',
       'BILL/LIM2', 'BILL/LIM3', 'BILL/LIM4', 'BILL/LIM5', 'BILL/LIM6',
       'AVE_BILL_AMT/LIM', 'AVE_PAY_AMT', 'PAY_AMT/LIM1', 'PAY_AMT/LIM2',
       'PAY_AMT/LIM3', 'PAY_AMT/LIM4', 'PAY_AMT/LIM5', 'PAY_AMT/LIM6',
       'AVE_PAY_AMT/LIM', 'PAY/BILL_1', 'PAY/BILL_2', 'PAY/BILL_3',
       'PAY/BILL_4', 'PAY/BILL_5', 'AVE_PAY/BILL', 'UNPAID_MONTHS'],
      dtype='object')